In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import pandas as pd
import re
#from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
import nltk.corpus as corpus
import nltk
import spacy
import math
import csv
nltk.download("brown")
pd.options.mode.chained_assignment = None  # default='warn'

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
def removePunctuation(s):
  out = re.sub("[^\w\s\u0300-\u036f]", "", s)
  out = re.sub("\d+", " ", out)
  out = re.sub(":", "", out)
  out = re.sub("[\\\n]", " ", out)
  out = re.sub(" +", " ", out)
  return out

# Vector semantics and embeddings

The idea of vector semantics is to use a point in a three-dimensional (or n-dimensional) space to present the meaning of a word. Vector semantics analisys makes the assumption that the meaning of a word can be defined by its distribution, meaning that two words with similar distributions also have similar meanings. The n-dimensional space is derived from the distriubution of the words.

In a mathematical sense an "embedding" is a mapping from one space or structure to another. By representing words as embeddings classifiers can assign sentiment as long as it sees words with similar meanings. The process can be done through automatic learning.

A vector space is a collection of vectors caracterized by their dimension.

Models:
 * ft-idf: The meaning of a word is defined by a simple function of the counts of other nearby words. Results in long and sparse (many zeroes) vectors.
 * Word2Vec: Short and dense vectors.
 * Cousine: Standard way to use embeddings.

 ## Term-document matrix
 In a term-document matrix each row represents a word in the vocabulary and each column represents the document from which the word have been extracted. This matrices are useful for document classification and information retrieval, as two similar documents tend to have similar words.

 The matrix is of shape |V| x |D|. |V| is the lenght of the vocabulary and |D| is the number of documents. By comparing two vectors we can get the idea of how similar two documents are, but it's possible to do the same with words.

 ## Term-term matrix
 Columns are labeled with words, menaning it's of shape |V| x |V|. It's better at comparing the similarity of words. Each cell records the number of times the row word (target) coocur in the context of a word in a column (context). The context is generally a window that takes all words within n "slots", meaning n words to the right and n to the left.

In [4]:
lem = spacy.load("en_core_web_sm")
window = 6
lim = 50000
text = " ".join(corpus.brown.words()[:lim])
text = lem(removePunctuation(text))
wlist = [i.lemma_ for i in text]
ttMatrix = pd.DataFrame(index = list(dict.fromkeys(wlist)), columns=list(dict.fromkeys(wlist)), data = 0)
for i in range(len(wlist)):
  for j in wlist[max((i-window, 0)):min((i+window, len(wlist)))]:
    ttMatrix[wlist[i]][j]+=1
  ttMatrix[wlist[i]][wlist[i]]=0

In [5]:
ttMatrix

,the,Fulton,County,Grand,Jury,say,Friday,an,investigation,of,...,Northwest,Expressways,crash,fiery,Snellville,Patrol,Claude,Maynor,Calvary,Troopers
the,0,17,21,1,1,249,20,85,9,1461,...,2,1,1,1,1,2,0,1,1,1
Fulton,15,0,6,1,1,2,2,1,0,3,...,0,0,0,0,0,0,0,0,0,0
County,20,6,0,1,1,5,2,1,1,13,...,0,0,0,0,0,0,0,0,0,0
Grand,1,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Jury,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patrol,2,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
Claude,0,0,0,0,0,1,0,0,0,2,...,0,0,0,0,0,0,0,1,1,1
Maynor,1,0,0,0,0,1,0,0,0,2,...,0,0,0,0,0,0,1,0,1,1
Calvary,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,1,0,1


In [6]:
import textwrap

In [7]:
print("Words: ", end = "")
print(textwrap.fill(", ".join(ttMatrix.columns), 200))

Words: the, Fulton, County, Grand, Jury, say, Friday, an, investigation, of, Atlantas, recent, primary, election, produce, no, evidence, that, any, irregularity, take, place, jury, far, in, termend,
presentment, City, Executive, Committee, which, have, overall, charge, deserve, praise, and, thank, Atlanta, for, manner, be, conduct, SeptemberOctober, term, by, Superior, Court, Judge, Durwood, Pye,
to, investigate, report, possible, hardfought, win, Mayornominate, Ivan, Allen, Jr, only, a, relative, handful, such, receive, consider, widespread, interest, number, voter, size, this, city, it, do,
find, many, georgias, registration, law, outmoded, or, inadequate, often, ambiguous, recommend, legislator, act, these, study, revise, end, modernize, improve, they, grand, comment, on, other, topic,
among, purchasing, department, well, operate, follow, generally, accept, practice, inure, good, both, government, Merger, propose, however, believe, two, office, should, combine, achieve, great,
effic

## Measuring similarity: Cosine

Cosine is the most common similarity metric, it takes two vectors of size len(V) x len(V) (len(V) x len(D) is term-document) and gives a measure of their similarity. It's based in the dot product. The dot product acts as a similarity metric because it'll tend to be high when two vectors have the same values in the same dimensions, but it favours long vectors, it's higher if the vector is longer. The dot product will be higher for frequent words. Because of that the vector must be normalized by dividing the dot product by the magnitude of both vectors. Vectors can be pre normalized by dividing by its magnitude.

This measure of similarity depends on the angle that is formed from both vectors, which is 0 if they are identical vectors or 90 if they are orthogonal vectors. Cosine can be used to measure the angle that two vectors form, similar to a normal 2dimensional angle, cosine tells how similar their direction is from 90º to 0, being 0 if it's 90 degrees or 1 if it's 0 degrees.

The cool thing about this is the fact that we can also make operations with this vectors, usually with Word2vec embeddings, to find or represent semantic relations. For example, the vector for the word "queen" is very close to the vector of the word "king" minus the vector for the word "man" plus the vector of the word "woman":

    V["queen"] =~ V["king"] - V["man"] + V["woman"]

This also shows biases in the language, with this operations it's possible to find relations that express gender or ethnic biases.

In [8]:
mod = lambda vec: np.sqrt(sum(vec**2))
cosine = lambda a, b: np.dot(a,b)/(mod(a)*mod(b))
cosine(ttMatrix["jury"], ttMatrix["law"])

0.8156650264774218

## TF-IDF: Weighting in terms of vectors

Raw frequence is not the best measure of association between words. Word types like determinants and pronouns don't add much to the discrimination. They are also very frequent. Words that occur nearby frequently are more important that words that occur rarely, but words that are too frequent are unimportant.

Tf-idf is used frequently for term-document, while PPMI is used for term-term.

Tf-idf weighting is the product of two terms. The first one is the frequency of the word *t* in the document *d*, a raw count, usually in a log10 space. The second term is used to give a higher weight to words that occur only in a few documents, terms that occur only in a few documents are more related to the topic and its content, the number of documents in which a word *t* occurs is called document frequency or df_t (not to be confused with collection frequency).

Idf accounts for Inverse document frequency (idf) and it's defined using the fraction N/df_t where N is the total number of documents. The fewer the documents the word occurs in the higher tha number. This term is also usually in a log10 space.

Tf-idf then is a weighting done to a particular word in a particular document as in: term_frequency(word, doc) * idf(word).

## Pointwise mutual information or PMI

It's an alternative weighting function used for term-term matrices. The best way to weight the association between two words is to ask how much more the two words co-occur in the corpus than we would expect. It's a measure of how often two events *x* and *y* occur compared to what we would expect if they were infependent:

    PMI(w,c) = log2( P(w, c) / (P(w) * P(c)) )

The numerator tells us how often we observed tha two words together. The denominator is how often we would expect the words to co-occur if they assuming they are idependent, the probability of two independent event co-occurring is the product of the probabilities of both events. This ratio is how much more the two words co-occur than we expect by chance.

PMI ranges from -1 to 1, but negative values are unreliable, thus we use PPMI or positive pointwise mutual information, so the PPMI is the max value between the PMI and 0:

    PPMI(w, c) = max(PMI(w, c), 0)

In [9]:
def ppmi(w: str,c: str, matrix: pd.DataFrame):
  count = sum(sum(matrix.to_numpy()))
  p_wc = ttMatrix[w][c]/count
  p_w = sum(ttMatrix.loc[w])/count
  p_c = sum(ttMatrix[c])/count
  pmi = p_wc/(p_c*p_w)
  return (max(pmi, 0), p_c, p_w, p_wc)

ppmi("jury", "law", ttMatrix)

(3.9501799520127965,
 0.0007423842344584391,
 0.0007191847271316129,
 2.1090461206205656e-06)

PMI has the problem of being biased towards very rare events, very rare words tend to have very high PMI values. One way to balance this is to slightly change the computation of P(c) with a weight *alpha* with a value of 0.75:

    Pa(c) = (count(c)^a)/sum(count(c)^a)

An alternative to this is Laplace smoothing.

##Word2Vec

Word2Vec is an algorithim that generates embeddings, which are short and dense vectors, dimensions varying between 50 and 1000 rather than the vocabulary size. Instead of sparse vectors embeddings are dense, meaning that they have much less zeroes, and the values inside these embeddings are real valued numbers that can be negative. This vectors work much better than the sparse, long ones. Word2vec methods are fast, efficient to train and easily available online.

Word2vec embeddings are static embeddings, meaning that the method learns one fixed embedding for each word. Instead of counting how often a word occurs near another it trains a classifier on a binary prediction task "is *w* likely to appear near *c*?" We don't care about this prediction, but we'll use the weights as the word embeddings. We can use running text as implicitly supervised training data for such a classifier: a word *c* that occurs near a target word *w* acts as gold to that question. This method is called self-supervision. It uses ogistic regression and simplifies the task by makin a binary clasification.

### Skip-gram with negative sampling or SGNS

SNGS is one of the two algorithms in the Word2vec software pack and it's sometimes referred as the Word2vec algorithm. The goal is to train a classifier such that, given a tuple (w, c) it'll return the probability that *c* is a real context word: P( + | w, c). the probability that *c* is not a real context word for *w* is: 1 - P( + | w, c).

The idea is to base this probability on embedding similarity, a word is likely to occur near words with similar embeddings, meaning we can use the dot product of the two embeddings, if the dot product is high it means they are similar. To turn the result in a probability we can use the sigmoid function, but for it to be a probability all probabilities must sum to one, meaning that P( + | w, c) = 1 - P( - | w, c). To calculate the probability of all context words in the window we make the assumption that all context words are independent, which means we can just multiply their probabilities.

Skipgram trains a probabilistic classifier that, given a target word and its context window, assigns a probability based on how similar this context window is to the target word. To compute this probability we just need the embeddings for each target and context word. Skipgram actually stores two embeddings fo each word, one of the word as a target and one of the word as a context word. Thus the paramepers we need to learn are the two matrices W and C, each containing an embedding for every word in the vocabulary.

